In [1]:
from signjoey.helpers import (
    load_config,
    load_checkpoint,
)
from signjoey.data import load_data
from signjoey.model import build_model


cfg_file = 'configs/example.yaml'
ckpt = 'output/best-kaggle.ckpt'

cfg = load_config(cfg_file)

In [2]:
model_checkpoint = load_checkpoint(ckpt, use_cuda=False)

do_recognition = cfg["training"].get("recognition_loss_weight", 1.0) > 0.0
do_translation = cfg["training"].get("translation_loss_weight", 1.0) > 0.0

_, dev_data, test_data, gls_vocab, txt_vocab = load_data(data_cfg=cfg["data"])

model = build_model(
    cfg=cfg["model"],
    gls_vocab=gls_vocab,
    txt_vocab=txt_vocab,
    sgn_dim=sum(cfg["data"]["feature_size"])
    if isinstance(cfg["data"]["feature_size"], list)
    else cfg["data"]["feature_size"],
    do_recognition=do_recognition,
    do_translation=do_translation,
    ensemble=False,
    ensembleN=1
)

d:\CS4203 - FYP\project folder\Signify\strf\lib\site-packages\torch\_utils.py:133: UserWarning: Failed to initialize NumPy: module compiled against API version 0x10 but this version of numpy is 0xf (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:68.)
  t = torch.tensor([], dtype=storage.dtype, device=storage._untyped().device)


512
512
512
512
512
512


In [117]:
import pickle
import gzip
filename = 'data/phoenix14t.pami0.train'


with gzip.open(filename, "rb") as f:
    loaded_object = pickle.load(f)

In [116]:
len(loaded_object[0]["sign"])

181

In [126]:
max_seq_d = loaded_object[0]
for d in loaded_object:
    if len(d["sign"])>400:
        continue
    if len(d["sign"])>len(max_seq_d['sign']):
        max_seq_d = d
print(max_seq_d)


{'name': 'train/29September_2010_Wednesday_heute-5458', 'signer': 'Signer07', 'gloss': 'MORGEN FRUEH FLUSS REGEN MORGEN S STEIN BIS SACHSEN REGION OFT SONNE TROCKEN FREUNDLICH DAFUER WEST ANFANG REGEN ABEND WAHRSCHEINLICH E REGION BIS ALLGAEU REGION REGEN', 'text': 'morgen scheint dann von schleswig holstein bis nach sachsen häufig die sonne es bleibt trocken während es im westen anfängt zu regnen und abends regnet es dann etwa so vom emsland bis zum allgäu .', 'sign': tensor([[0.0732, 0.2189, 0.3328,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 1.0489, 0.4065,  ..., 0.3054, 0.0000, 0.3405],
        [0.0322, 1.0775, 0.3299,  ..., 0.6737, 0.0000, 0.9411],
        ...,
        [0.0000, 0.2249, 0.6509,  ..., 0.7619, 0.0000, 0.3435],
        [0.0000, 0.5764, 0.8349,  ..., 0.7944, 0.0000, 0.0497],
        [0.0000, 0.3482, 0.6576,  ..., 1.8677, 0.0000, 0.2705]])}


In [127]:
max_seq_d['sign'].shape

torch.Size([395, 1024])

In [128]:
loaded_object[0]['sign'].shape

torch.Size([86, 1024])

In [39]:
loaded_object[0]['sign'][0][0].dtype

torch.float32

In [57]:

dev_data[0].__dict__.values().mapping

mappingproxy({'sequence': 'dev/11August_2010_Wednesday_tagesschau-2',
              'signer': 'Signer08',
              'sgn': [tensor([4.1970e-01, 1.7552e+00, 2.4212e-01,  ..., 1.0000e-08, 1.0000e-08,
                       1.0000e-08]),
               tensor([2.3592e+00, 2.2639e+00, 4.4332e-01,  ..., 1.0000e-08, 1.0000e-08,
                       1.0000e-08]),
               tensor([5.0251e+00, 2.1903e+00, 1.3583e-01,  ..., 5.7079e-02, 1.0000e-08,
                       1.0000e-08]),
               tensor([5.3371e+00, 1.3213e+00, 1.0000e-08,  ..., 2.1011e+00, 1.0000e-08,
                       1.0000e-08]),
               tensor([4.2258e+00, 2.5479e+00, 1.0000e-08,  ..., 8.2129e-01, 1.0000e-08,
                       1.0000e-08]),
               tensor([3.6651e+00, 3.7671e+00, 1.0000e-08,  ..., 3.9290e-01, 1.0000e-08,
                       1.0818e-01]),
               tensor([3.1189e+00, 3.1486e+00, 1.0000e-08,  ..., 1.8473e-01, 1.0000e-08,
                       1.0979e+00]),
     

In [40]:
dev_data.fields

{'sequence': <torchtext.data.field.RawField at 0x1a9f7e0e1a0>,
 'signer': <torchtext.data.field.RawField at 0x1a9f7e23400>,
 'sgn': <torchtext.data.field.Field at 0x1a9f7e20ac0>,
 'gls': <torchtext.data.field.Field at 0x1a9f7e21f60>,
 'txt': <torchtext.data.field.Field at 0x1a9f7e20970>}

In [3]:
model

SignModel(
	encoder=TransformerEncoder(num_layers=2, num_heads=8),
	decoder=TransformerDecoder(num_layers=2, num_heads=8),
	sgn_embed=SpatialEmbeddings(embedding_dim=512, input_size=1024),
	txt_embed=Embeddings(embedding_dim=512, vocab_size=2892))

In [4]:
model.load_state_dict(model_checkpoint["model_state"])

<All keys matched successfully>

In [27]:
model.sgn_embed

SpatialEmbeddings(embedding_dim=512, input_size=1024)

In [5]:
model.encoder

TransformerEncoder(num_layers=2, num_heads=8)

In [6]:
model.parameters

<bound method Module.parameters of SignModel(
	encoder=TransformerEncoder(num_layers=2, num_heads=8),
	decoder=TransformerDecoder(num_layers=2, num_heads=8),
	sgn_embed=SpatialEmbeddings(embedding_dim=512, input_size=1024),
	txt_embed=Embeddings(embedding_dim=512, vocab_size=2892))>

In [7]:
for name, param in model.named_parameters():
    print(f"Layer: {name} | Shape: {param.shape} | Requires Grad: {param.requires_grad}")

Layer: encoder.layers.0.layer_norm.weight | Shape: torch.Size([512]) | Requires Grad: True
Layer: encoder.layers.0.layer_norm.bias | Shape: torch.Size([512]) | Requires Grad: True
Layer: encoder.layers.0.src_src_att.k_layer.posterior_mean | Shape: torch.Size([512, 512]) | Requires Grad: True
Layer: encoder.layers.0.src_src_att.k_layer.posterior_un_scale | Shape: torch.Size([512, 512]) | Requires Grad: True
Layer: encoder.layers.0.src_src_att.k_layer.bias_mean | Shape: torch.Size([512]) | Requires Grad: True
Layer: encoder.layers.0.src_src_att.k_layer.bias_un_scale | Shape: torch.Size([512]) | Requires Grad: True
Layer: encoder.layers.0.src_src_att.v_layer.posterior_mean | Shape: torch.Size([512, 512]) | Requires Grad: True
Layer: encoder.layers.0.src_src_att.v_layer.posterior_un_scale | Shape: torch.Size([512, 512]) | Requires Grad: True
Layer: encoder.layers.0.src_src_att.v_layer.bias_mean | Shape: torch.Size([512]) | Requires Grad: True
Layer: encoder.layers.0.src_src_att.v_layer.bia

In [8]:
dev_data[0].__dict__["sgn"]

[tensor([4.1970e-01, 1.7552e+00, 2.4212e-01,  ..., 1.0000e-08, 1.0000e-08,
         1.0000e-08]),
 tensor([2.3592e+00, 2.2639e+00, 4.4332e-01,  ..., 1.0000e-08, 1.0000e-08,
         1.0000e-08]),
 tensor([5.0251e+00, 2.1903e+00, 1.3583e-01,  ..., 5.7079e-02, 1.0000e-08,
         1.0000e-08]),
 tensor([5.3371e+00, 1.3213e+00, 1.0000e-08,  ..., 2.1011e+00, 1.0000e-08,
         1.0000e-08]),
 tensor([4.2258e+00, 2.5479e+00, 1.0000e-08,  ..., 8.2129e-01, 1.0000e-08,
         1.0000e-08]),
 tensor([3.6651e+00, 3.7671e+00, 1.0000e-08,  ..., 3.9290e-01, 1.0000e-08,
         1.0818e-01]),
 tensor([3.1189e+00, 3.1486e+00, 1.0000e-08,  ..., 1.8473e-01, 1.0000e-08,
         1.0979e+00]),
 tensor([4.0009e+00, 3.6177e+00, 1.0000e-08,  ..., 1.0000e-08, 1.0000e-08,
         2.3070e+00]),
 tensor([4.9496e+00, 3.4256e+00, 1.0000e-08,  ..., 1.0000e-08, 1.0000e-08,
         2.5931e+00]),
 tensor([6.0652e+00, 3.4525e+00, 4.0531e-02,  ..., 1.0000e-08, 1.0000e-08,
         2.8201e+00]),
 tensor([5.3373e+00,

In [14]:
from signjoey.data import load_data, make_data_iter
from signjoey.batch import Batch

valid_iter = make_data_iter(
    dataset=dev_data,
    batch_size=32,
    batch_type="word",
    shuffle=False,
    train=False,
)

In [15]:
first_batch = None
for valid_batch in iter(valid_iter):
    # print(valid_batch.sgn)
    # print(len(valid_batch.sgn[0]), len(valid_batch.sgn[1]))
    # print(valid_batch.sgn[0][0], valid_batch.sgn[1][0] , len(valid_batch.sgn[0][0]))
    first_batch = valid_batch
    print(valid_batch)
    break
    
    # batch = Batch(
    #     is_train=False,
    #     torch_batch=valid_batch,
    #     txt_pad_index=txt_pad_index,
    #     sgn_dim=sgn_dim,
    #     use_cuda=use_cuda,
    #     frame_subsampling_ratio=frame_subsampling_ratio,
    # )


[torchtext.data.batch.Batch of size 32]
	[.sequence]:['dev/11August_2010_Wednesday_tagesschau-2', 'dev/11August_2010_Wednesday_tagesschau-3', 'dev/11August_2010_Wednesday_tagesschau-8', 'dev/25October_2010_Monday_tagesschau-22', 'dev/05May_2011_Thursday_tagesschau-25', 'dev/15December_2010_Wednesday_tagesschau-40', 'dev/10March_2011_Thursday_heute-51', 'dev/14August_2009_Friday_tagesschau-72', 'dev/26January_2010_Tuesday_heute-107', 'dev/12February_2010_Friday_tagesschau-125', 'dev/12February_2010_Friday_tagesschau-126', 'dev/01November_2010_Monday_tagesschau-140', 'dev/28November_2011_Monday_tagesschau-165', 'dev/28November_2011_Monday_tagesschau-168', 'dev/28November_2011_Monday_tagesschau-172', 'dev/24November_2011_Thursday_heute-222', 'dev/24November_2011_Thursday_heute-223', 'dev/19October_2009_Monday_tagesschau-229', 'dev/12July_2010_Monday_heute-244', 'dev/21August_2011_Sunday_tagesschau-264', 'dev/21August_2011_Sunday_tagesschau-265', 'dev/22November_2010_Monday_heute-279', 'd

d:\CS4203 - FYP\project folder\Signify\strf\lib\site-packages\torchtext\data\field.py:359: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  var = torch.tensor(arr, dtype=self.dtype, device=device)


In [90]:
len(first_batch.sgn[1])

32

In [17]:
first_batch.sgn[0]

tensor([[[4.1970e-01, 1.7552e+00, 2.4212e-01,  ..., 1.0000e-08,
          1.0000e-08, 1.0000e-08],
         [2.3592e+00, 2.2639e+00, 4.4332e-01,  ..., 1.0000e-08,
          1.0000e-08, 1.0000e-08],
         [5.0251e+00, 2.1903e+00, 1.3583e-01,  ..., 5.7079e-02,
          1.0000e-08, 1.0000e-08],
         ...,
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00]],

        [[1.0000e-08, 1.0000e-08, 1.0000e-08,  ..., 1.0000e-08,
          1.0000e-08, 1.0000e-08],
         [1.0000e-08, 1.0000e-08, 1.4179e+00,  ..., 1.0000e-08,
          1.0000e-08, 1.0000e-08],
         [4.3050e-03, 1.0000e-08, 2.1061e+00,  ..., 1.0000e-08,
          1.0000e-08, 1.0000e-08],
         ...,
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
          0.000

In [108]:
len(first_batch.sgn[0][0])

165

In [99]:
first_batch.txt[1]

tensor([12, 16, 21, 17, 13, 13, 30, 17, 11, 17, 15, 13, 14, 19, 13, 19, 28, 13,
        19, 10, 18, 28, 16,  9, 11, 16, 12, 14, 26, 16, 18, 11])

In [25]:
first_batch.sgn[0]

tensor([[[4.1970e-01, 1.7552e+00, 2.4212e-01,  ..., 1.0000e-08,
          1.0000e-08, 1.0000e-08],
         [2.3592e+00, 2.2639e+00, 4.4332e-01,  ..., 1.0000e-08,
          1.0000e-08, 1.0000e-08],
         [5.0251e+00, 2.1903e+00, 1.3583e-01,  ..., 5.7079e-02,
          1.0000e-08, 1.0000e-08],
         ...,
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00]],

        [[1.0000e-08, 1.0000e-08, 1.0000e-08,  ..., 1.0000e-08,
          1.0000e-08, 1.0000e-08],
         [1.0000e-08, 1.0000e-08, 1.4179e+00,  ..., 1.0000e-08,
          1.0000e-08, 1.0000e-08],
         [4.3050e-03, 1.0000e-08, 2.1061e+00,  ..., 1.0000e-08,
          1.0000e-08, 1.0000e-08],
         ...,
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
          0.000

In [ ]:
batch = Batch(
    is_train=False,
    torch_batch=first_batch,
    txt_pad_index=txt_pad_index,
    sgn_dim=sgn_dim,
    use_cuda=use_cuda,
    frame_subsampling_ratio=frame_subsampling_ratio,
)

In [24]:
encoder_output, encoder_hidden = model.encode(
                sgn=first_batch.sgn[0], sgn_mask=None, sgn_length=None
            )

AttributeError: 'NoneType' object has no attribute 'reshape'

In [29]:
model.sgn_embed(first_batch.sgn[0], mask=None)

AttributeError: 'NoneType' object has no attribute 'reshape'

# Data Loaders

In [ ]:
from signjoey.helpers import (
    load_config,
    load_checkpoint,
)
from signjoey.data import load_data
from signjoey.model import build_model


cfg_file = 'configs/example.yaml'
ckpt = 'output/best-kaggle.ckpt'

cfg = load_config(cfg_file)

In [ ]:
train_data, dev_data, test_data, gls_vocab, txt_vocab = load_data(data_cfg=cfg["data"])